In [ ]:
from torchvision import transforms
import matplotlib.pyplot as plt
from src.models.utils.data import FoodPricingDataset, FoodPricingLazyDataset
import torch
from itertools import islice

In [ ]:
img_transform = transforms.Compose(
    [
        transforms.Resize(size=(224, 224)),
        transforms.ToTensor(),
        # transforms.Normalize(
        #     mean=[0.485, 0.456, 0.406],
        #     std=[0.229, 0.224, 0.225],
        # )
    ]
)

In [ ]:
training_data = FoodPricingDataset(
    img_transform=img_transform, txt_transform=lambda x: x, split="train"
)

figure = plt.figure(figsize=(8, 8))
cols, rows = 3, 3
for i in range(1, cols * rows + 1):
    sample_idx = torch.randint(len(training_data), size=(1,)).item()
    sample = training_data[sample_idx]
    figure.add_subplot(rows, cols, i)
    plt.title(f"{sample['label'].numpy()[0] / 100:,.2f}€")
    plt.axis("off")
    plt.imshow(sample["img"].permute(1, 2, 0))
plt.show()

In [ ]:
for i in islice(training_data.iter_txt(), 5):
    print(i)

In [ ]:
lazy_training_data = FoodPricingLazyDataset(
    img_transform=img_transform, txt_transform=lambda x: x, split="train"
)

figure = plt.figure(figsize=(8, 8))
cols, rows = 3, 3
for i in range(1, cols * rows + 1):
    sample_idx = torch.randint(len(training_data), size=(1,)).item()
    sample = training_data[sample_idx]
    figure.add_subplot(rows, cols, i)
    plt.title(f"{sample['label'].numpy()[0] / 100:,.2f}€")
    plt.axis("off")
    plt.imshow(sample["img"].permute(1, 2, 0))
plt.show()

In [ ]:
for i in islice(lazy_training_data.iter_txt(), 5):
    print(i)